In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install emoji
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 17.7 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=ba20df3c6628bde9c7e158da85497bafa0c0563c5734f307fa7e2e043beeb15e
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 24.1 MB/s 


In [3]:
#import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.svm import SVC
import string
import unidecode
import emoji
import nltk
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
data = pd.read_csv('/content/drive/MyDrive/NLP2/data/train_all_tasks.csv')

In [5]:
data

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,"""GRAPHIC Germany - Muslim ""refugee"" stabbing h...",not sexist,none,none
13997,sexism2022_english-3320,Lol I imagine there would be simps that are li...,not sexist,none,none
13998,sexism2022_english-13309,"It's not, the girls I go on dates with don't k...",not sexist,none,none


In [6]:
def preprocess(text):
  text = text.lower()
  text = re.sub( r"(\?+)", r" twinterrogation", text)
  text = re.sub( r"(!+|¡+)", r" twexclamation", text)
  text = re.sub( r"(?<=^|(?<=[^a-zA-Z0-9-\\.]))#([A-Za-z]+[A-Za-z0-9]+)", r"twhastag", text)


  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r'', text)
  text = re.sub('[^a-zA-Z]'," ",text)

  tokenizer = TweetTokenizer()
  stop_words_nltk = set(stopwords.words('english'))
  stemmer = PorterStemmer() 

  words = tokenizer.tokenize(text)
  filtered_words = [word for word in words if word not in stop_words_nltk]
  stemmed_words = [stemmer.stem(word) for word in filtered_words]
  return ' '.join(stemmed_words)

In [7]:
data['text'] = data['text'].apply(preprocess)

In [8]:
data

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,damn write pretti chaotic,not sexist,none,none
1,sexism2022_english-2367,yeah appar bunch misogynist virgin one turn ga...,not sexist,none,none
2,sexism2022_english-3073,fuck woman still mp twexclam twinterrog,not sexist,none,none
3,sexism2022_english-14895,understand know right time know enough money w...,not sexist,none,none
4,sexism2022_english-4118,surpriz stop rape women,not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,complex like nice chick go bad boy nah nah nah...,sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,graphic germani muslim refuge stab younger yo ...,not sexist,none,none
13997,sexism2022_english-3320,lol imagin would simp like deserv ugli guy jfl,not sexist,none,none
13998,sexism2022_english-13309,girl go date kiss first date text back first date,not sexist,none,none


In [ ]:
train, test = train_test_split(data, test_size=0.2)

# Task 1


TF IDF + SVM

In [ ]:
le = LabelEncoder()

X_train, y_train = train['text'], train['label_sexist']
y_train = le.fit_transform(y_train)
print("Labels: {}".format(dict(zip(le.classes_, le.transform(le.classes_)))))

X_test, y_test = test['text'], test['label_sexist']
y_test = le.fit_transform(y_test)

#hyperparameter tuning using TFIDF

max_features_tfidf_param = [3000, 5000, 8000]
C_param = [1.0, 1.5, 2.0, 2.5, 3.0]
kernel_param = ['rbf', 'linear']

best_parameters = {}
best_parameters['best_acc'] = 0

for max_features in max_features_tfidf_param:
  #initiate the TfidfVectorizer
  tfidf = TfidfVectorizer(max_features=max_features)

  #convert the training and validation data to tf-idf indexes
  X_train_tfidf = tfidf.fit_transform(X_train).toarray()
  X_test_tfidf = tfidf.transform(X_test).toarray()
  
  for C in C_param:
    for kernel in kernel_param:
      model = SVC(C=C, kernel=kernel)
      model.fit(X_train_tfidf, y_train)
      y_pred = model.predict(X_test_tfidf)
      score = accuracy_score(y_pred, y_test)
      print('Params: TFIDF max features={}, C={},  Kernel={}, Acc: {}'.format(max_features, C, kernel, score))

      #save the parameters for the best score
      if score > best_parameters['best_acc']:
        best_parameters['best_acc'] = score
        best_parameters["max_features"] = max_features
        best_parameters["C"] = C
        best_parameters['kernel'] = kernel_param


print(best_parameters)

Labels: {'not sexist': 0, 'sexist': 1}
Params: TFIDF max features=3000, C=1.0,  Kernel=rbf, Acc: 0.8296428571428571
Params: TFIDF max features=3000, C=1.0,  Kernel=linear, Acc: 0.8328571428571429
Params: TFIDF max features=3000, C=1.5,  Kernel=rbf, Acc: 0.8296428571428571
Params: TFIDF max features=3000, C=1.5,  Kernel=linear, Acc: 0.8314285714285714
Params: TFIDF max features=3000, C=2.0,  Kernel=rbf, Acc: 0.8260714285714286
Params: TFIDF max features=3000, C=2.0,  Kernel=linear, Acc: 0.8289285714285715
Params: TFIDF max features=3000, C=2.5,  Kernel=rbf, Acc: 0.825
Params: TFIDF max features=3000, C=2.5,  Kernel=linear, Acc: 0.8225
Params: TFIDF max features=3000, C=3.0,  Kernel=rbf, Acc: 0.825
Params: TFIDF max features=3000, C=3.0,  Kernel=linear, Acc: 0.8192857142857143
Params: TFIDF max features=5000, C=1.0,  Kernel=rbf, Acc: 0.8271428571428572
Params: TFIDF max features=5000, C=1.0,  Kernel=linear, Acc: 0.8321428571428572


TIF IDF + LR


In [ ]:
from sklearn.linear_model import LogisticRegression
le = LabelEncoder()
X_train, y_train = train['text'], train['label_sexist']
y_train = le.fit_transform(y_train)
print("Labels: {}".format(dict(zip(le.classes_, le.transform(le.classes_)))))

X_test, y_test = test['text'], test['label_sexist']
y_test = le.fit_transform(y_test)

#hyperparameter tuning using TFIDF

max_features_tfidf_param = [3000, 5000, 8000]
penalty_param = ['l2', 'none']

best_parameters = {}
best_parameters['best_acc'] = 0

for max_features in max_features_tfidf_param:
  #initiate the TfidfVectorizer
  tfidf = TfidfVectorizer(max_features=max_features)

  #convert the training and validation data to tf-idf indexes
  X_train_tfidf = tfidf.fit_transform(X_train).toarray()
  X_test_tfidf = tfidf.transform(X_test).toarray()

  for penalty in penalty_param:
    model = LogisticRegression(penalty=penalty, max_iter = 100)
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    score = accuracy_score(y_pred, y_test)
    print('Params: TFIDF max features={}, penalty={}, Acc: {}'.format(max_features, penalty, score))

    #save the parameters for the best score
    if score > best_parameters['best_acc']:
      best_parameters['best_acc'] = score
      best_parameters["max_features"] = max_features
      best_parameters["penalty"] = penalty


print(best_parameters)

Labels: {'not sexist': 0, 'sexist': 1}
Params: TFIDF max features=3000, penalty=l2, Acc: 0.8328571428571429


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Params: TFIDF max features=3000, penalty=none, Acc: 0.7785714285714286
Params: TFIDF max features=5000, penalty=l2, Acc: 0.8332142857142857


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Params: TFIDF max features=5000, penalty=none, Acc: 0.7596428571428572
Params: TFIDF max features=8000, penalty=l2, Acc: 0.8325
Params: TFIDF max features=8000, penalty=none, Acc: 0.7742857142857142
{'best_acc': 0.8332142857142857, 'max_features': 5000, 'penalty': 'l2'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# Task 2

TFIDF + SVM

In [13]:
data_task2 = data[data['label_sexist'] == 'sexist']
train, test = train_test_split(data_task2, test_size=0.2)

In [14]:
le = LabelEncoder()

X_train, y_train = train['text'], train['label_category']
y_train = le.fit_transform(y_train)
print("Labels: {}".format(dict(zip(le.classes_, le.transform(le.classes_)))))

X_test, y_test = test['text'], test['label_category']
y_test = le.fit_transform(y_test)

#hyperparameter tuning using TFIDF

max_features_tfidf_param = [3000, 5000, 8000]
C_param = [1.0, 2.0, 3.0]
kernel_param = ['rbf', 'linear']

best_parameters = {}
best_parameters['best_acc'] = 0

for max_features in max_features_tfidf_param:
  #initiate the TfidfVectorizer
  tfidf = TfidfVectorizer(max_features=max_features)

  #convert the training and validation data to tf-idf indexes
  X_train_tfidf = tfidf.fit_transform(X_train).toarray()
  X_test_tfidf = tfidf.transform(X_test).toarray()
  
  for C in C_param:
    for kernel in kernel_param:
      model = SVC(C=C, kernel=kernel)
      model.fit(X_train_tfidf, y_train)
      y_pred = model.predict(X_test_tfidf)
      score = accuracy_score(y_pred, y_test)
      print('Params: TFIDF max features={}, C={},  Kernel={}, Acc: {}'.format(max_features, C, kernel, score))

      #save the parameters for the best score
      if score > best_parameters['best_acc']:
        best_parameters['best_acc'] = score
        best_parameters["max_features"] = max_features
        best_parameters["C"] = C
        best_parameters['kernel'] = kernel_param


print(best_parameters)

Labels: {'1. threats, plans to harm and incitement': 0, '2. derogation': 1, '3. animosity': 2, '4. prejudiced discussions': 3}
Params: TFIDF max features=3000, C=1.0,  Kernel=rbf, Acc: 0.5088235294117647
Params: TFIDF max features=3000, C=1.0,  Kernel=linear, Acc: 0.5029411764705882
Params: TFIDF max features=3000, C=2.0,  Kernel=rbf, Acc: 0.4970588235294118
Params: TFIDF max features=3000, C=2.0,  Kernel=linear, Acc: 0.48823529411764705
Params: TFIDF max features=3000, C=3.0,  Kernel=rbf, Acc: 0.4970588235294118
Params: TFIDF max features=3000, C=3.0,  Kernel=linear, Acc: 0.47352941176470587
Params: TFIDF max features=5000, C=1.0,  Kernel=rbf, Acc: 0.5073529411764706
Params: TFIDF max features=5000, C=1.0,  Kernel=linear, Acc: 0.5117647058823529
Params: TFIDF max features=5000, C=2.0,  Kernel=rbf, Acc: 0.5029411764705882
Params: TFIDF max features=5000, C=2.0,  Kernel=linear, Acc: 0.48823529411764705
Params: TFIDF max features=5000, C=3.0,  Kernel=rbf, Acc: 0.49264705882352944
Params:

TFIDF + LR

In [15]:
from sklearn.linear_model import LogisticRegression
le = LabelEncoder()
X_train, y_train = train['text'], train['label_category']
y_train = le.fit_transform(y_train)
print("Labels: {}".format(dict(zip(le.classes_, le.transform(le.classes_)))))

X_test, y_test = test['text'], test['label_category']
y_test = le.fit_transform(y_test)

#hyperparameter tuning using TFIDF

max_features_tfidf_param = [3000, 5000, 8000]
penalty_param = ['l2', 'none']

best_parameters = {}
best_parameters['best_acc'] = 0

for max_features in max_features_tfidf_param:
  #initiate the TfidfVectorizer
  tfidf = TfidfVectorizer(max_features=max_features)

  #convert the training and validation data to tf-idf indexes
  X_train_tfidf = tfidf.fit_transform(X_train).toarray()
  X_test_tfidf = tfidf.transform(X_test).toarray()

  for penalty in penalty_param:
    model = LogisticRegression(penalty=penalty, max_iter = 100)
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    score = accuracy_score(y_pred, y_test)
    print('Params: TFIDF max features={}, penalty={}, Acc: {}'.format(max_features, penalty, score))

    #save the parameters for the best score
    if score > best_parameters['best_acc']:
      best_parameters['best_acc'] = score
      best_parameters["max_features"] = max_features
      best_parameters["penalty"] = penalty


print(best_parameters)

Labels: {'1. threats, plans to harm and incitement': 0, '2. derogation': 1, '3. animosity': 2, '4. prejudiced discussions': 3}
Params: TFIDF max features=3000, penalty=l2, Acc: 0.5029411764705882
Params: TFIDF max features=3000, penalty=none, Acc: 0.4647058823529412
Params: TFIDF max features=5000, penalty=l2, Acc: 0.4970588235294118
Params: TFIDF max features=5000, penalty=none, Acc: 0.4661764705882353
Params: TFIDF max features=8000, penalty=l2, Acc: 0.49117647058823527
Params: TFIDF max features=8000, penalty=none, Acc: 0.475
{'best_acc': 0.5029411764705882, 'max_features': 3000, 'penalty': 'l2'}
